<a href="https://colab.research.google.com/github/Anurag131277/langchain/blob/main/SwaggerChat_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --quiet -U langgraph langchain_community langchain_core langchain_nvidia_ai_endpoints langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.4/423.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 10.6 MB/s eta 0:00:00


In [2]:

import os
from google.colab import userdata

In [3]:

os.environ["NVIDIA_API_KEY"] = userdata.get('NVIDIA_API_KEY')

In [4]:
llm_str= 'meta/llama-3.3-70b-instruct'

In [46]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
# from typing import Any

# from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
# from langchain_community.utilities.openapi import OpenAPISpec
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_community.tools.json.tool import JsonSpec
# from langchain.chains import APIChain
# from langchain_community.agent_toolkits import JsonToolkit,create_json_agent
# from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

import json
import argparse
import requests

from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

ALLOW_DANGEROUS_REQUEST = True


In [59]:
# url = 'https://petstore.swagger.io/v2/swagger.json'
# url3='https://petstore31.swagger.io/api/v31/openapi.json'
# response= requests.get(url3)

# payload= response.json()
# print(str(payload))


with open("open_ai.json") as f:
    openapi = json.load(f)

In [60]:
# llm = ChatNVIDIA(model=llm_str,temperature=0)
# parsed_spec=OpenAPISpec.from_url(url3)

requests_wrapper = RequestsWrapper()
# json_spec = JsonSpec(dict_=payload, max_value_length=4000)
# openai_api_spec = reduce_openapi_spec(json_spec)
# print(json_spec)

In [61]:
# openai_fns, call_api_fn = openapi_spec_to_openai_fn(json_spec)
# json_toolkit = JsonToolkit(spec=json_spec)

# json_agent_executor = create_json_agent(
#     llm=ChatNVIDIA(model=llm_str,temperature=0), toolkit=json_toolkit, verbose=True
# )

f1_spec = reduce_openapi_spec(openapi,dereference=False)

In [32]:
# prompt = ChatPromptTemplate.from_template(
#     "Use the provided APIs to respond to this user query:\n\n{query}"
# )

In [62]:
llm = ChatNVIDIA(model=llm_str,temperature=0)

In [63]:
f1_agent = planner.create_openapi_agent(
    f1_spec, requests_wrapper, llm, allow_dangerous_requests=True
)

In [65]:
user_query = "what is the name of the pet with Id 1"
response = f1_agent.run(user_query)
print(f"response ${response}")



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get the name of the pet with Id 1
Observation: To find the name of the pet with Id 1, we can use the GET endpoint to retrieve the pet's information.

Plan: 
1. GET /pet/1 to retrieve the pet's information, which should include the name of the pet.

This single API call should provide the necessary information to solve the user's query.
Thought:I have a plan to retrieve the pet's information, now I should execute this plan using the api_controller tool to get the actual data.

Action: api_controller
Action Input: 1. GET /pet/1 to retrieve the pet's information

> Entering new AgentExecutor chain...
To execute the plan, I need to start by retrieving the pet's information using the GET /pet/1 endpoint. This endpoint requires the petId as a path parameter, which in this case is 1.

Action: requests_get
Action Input: {"url": "https://petstore31.swagger.io/api/v31/pet/1", "params":